# Backtesting Ratings With Historical Stream Data

Betfair's historical stream data is an extremely rich source of data. However, in it's raw form it's difficult to handle for the uninitiated. It also might not be immediately obvious how many different things this dataset could be used for without seeing some examples. These guides will hopefully demystify how to turn this raw data into a familiar and usable format whilst also hopefully providing some inpiration for the kinds of gold that can be excavated from it.

In [92]:
import pandas
import os
import re
import betfairlightweight
import logging


# setup logging
logging.basicConfig(level=logging.INFO)

In [32]:
data_path = [
    "./data/2021_01_JanRacingPro.tar",
    "./data/2021_01_JanRacingPro.tar"
]

stream_directory = "./data/"

In [62]:
def load_markets(file_paths):
    for file_path in file_paths:
        print(file_path)
        ext = os.path.splitext(file_path)[1]
        # iterate through a tar archive
        if ext == '.tar':
            with tarfile.TarFile(file_path) as archive:
                for file in archive:
                    yield bz2.open(archive.extractfile(file))
        # or a zip archive
        elif ext == '.zip':
            with zipfile.ZipFile(file_path) as archive:
                for file in archive.namelist():
                    yield bz2.open(archive.open(file))

    return None

In [63]:
for file_obj in load_markets(data_path):
    stream = trading.streaming.create_historical_generator_stream(
        file_path=file_obj,
        listener=listener,
    )

    with patch("builtins.open", lambda f, _: f): 

SyntaxError: unexpected EOF while parsing (<ipython-input-63-6b508a70ef4f>, line 7)

In [80]:
# Following the exmaple from betfair light weight
file_objects = load_markets(data_path)

trading = betfairlightweight.APIClient("username", "password")

listener = betfairlightweight.StreamListener(max_latency=None)

In [84]:
file_objects

<generator object load_markets at 0x0000022BF63F2CF0>

In [95]:
stream = trading.streaming.create_historical_generator_stream(
            file_path="./data/example-market",
            listener=listener,
        )

# create generator
gen = stream.get_generator()

In [99]:
# print marketBooks
for market_books in gen():
    for market_book in market_books:
        print(market_book)

INFO:betfairlightweight.streaming.listener:[Register: 0]: marketSubscription
INFO:betfairlightweight.streaming.stream:[MarketStream: 0]: "MarketStream" created
INFO:betfairlightweight.streaming.stream:[MarketStream: 0]: 1.179877785 added, 1 markets in cache


MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

...

MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook
MarketBook

In [100]:
# print based on seconds to start
for market_books in gen():
    for market_book in market_books:
        seconds_to_start = (
            market_book.market_definition.market_time - market_book.publish_time
        ).total_seconds()
        if seconds_to_start < 100:
            print(market_book.market_id, seconds_to_start, market_book.total_matched)

        # print winner details once market is closed
        if market_book.status == "CLOSED":
            for runner in market_book.runners:
                if runner.status == "WINNER":
                    print(
                        "{0}: {1} with sp of {2}".format(
                            runner.status, runner.selection_id, runner.sp.actual_sp
                        )
                    )

INFO:betfairlightweight.streaming.listener:[Register: 0]: marketSubscription
INFO:betfairlightweight.streaming.stream:[MarketStream: 0]: "MarketStream" created
INFO:betfairlightweight.streaming.stream:[MarketStream: 0]: 1.179877785 added, 1 markets in cache


1.179877785 99.951 0
1.179877785 99.727 0
1.179877785 99.642 0
1.179877785 99.534 0
1.179877785 99.45 0
1.179877785 99.333 0
1.179877785 99.247 0
1.179877785 99.134 0
1.179877785 99.044 0
1.179877785 98.938 0
1.179877785 98.836 0
1.179877785 98.767 0
1.179877785 98.658 0
1.179877785 98.552 0
1.179877785 98.44 0
1.179877785 98.321 0
1.179877785 98.228 0
1.179877785 98.125 0
1.179877785 98.037 0
1.179877785 97.959 0
1.179877785 97.845 0
1.179877785 97.745 0
1.179877785 97.645 0
1.179877785 97.539 0
1.179877785 97.439 0
1.179877785 97.328 0
1.179877785 97.216 0
1.179877785 97.133 0
1.179877785 97.03 0
1.179877785 96.975 0
1.179877785 96.868 0
1.179877785 96.763 0
1.179877785 96.652 0
1.179877785 96.547 0
1.179877785 96.447 0
1.179877785 96.336 0
1.179877785 96.23 0
1.179877785 96.18 0
1.179877785 96.068 0
1.179877785 95.961 0
1.179877785 95.857 0
1.179877785 95.749 0
1.179877785 95.644 0
1.179877785 95.536 0
1.179877785 95.432 0
1.179877785 95.331 0
1.179877785 95.255 0
1.179877785 95.157

1.179877785 34.75 0
1.179877785 34.643 0
1.179877785 34.589 0
1.179877785 34.483 0
1.179877785 34.373 0
1.179877785 34.27 0
1.179877785 34.13 0
1.179877785 33.845 0
1.179877785 33.746 0
1.179877785 33.676 0
1.179877785 33.353 0
1.179877785 33.248 0
1.179877785 33.144 0
1.179877785 33.041 0
1.179877785 32.989 0
1.179877785 32.879 0
1.179877785 32.768 0
1.179877785 32.665 0
1.179877785 32.556 0
1.179877785 32.451 0
1.179877785 32.344 0
1.179877785 32.244 0
1.179877785 32.178 0
1.179877785 32.078 0
1.179877785 31.956 0
1.179877785 31.755 0
1.179877785 31.53 0
1.179877785 31.504 0
1.179877785 31.337 0
1.179877785 31.28 0
1.179877785 31.178 0
1.179877785 31.068 0
1.179877785 30.964 0
1.179877785 30.86 0
1.179877785 30.74 0
1.179877785 30.642 0
1.179877785 30.58 0
1.179877785 30.474 0
1.179877785 30.369 0
1.179877785 30.255 0
1.179877785 30.158 0
1.179877785 30.051 0
1.179877785 29.949 0
1.179877785 29.682 0
1.179877785 29.638 0
1.179877785 29.609 0
1.179877785 29.55 0
1.179877785 29.461 0
1

1.179877785 -15.539 0
1.179877785 -15.591 0
1.179877785 -15.696 0
1.179877785 -15.805 0
1.179877785 -15.904 0
1.179877785 -16.005 0
1.179877785 -16.119 0
1.179877785 -16.172 0
1.179877785 -16.284 0
1.179877785 -16.513 0
1.179877785 -16.615 0
1.179877785 -16.715 0
1.179877785 -16.825 0
1.179877785 -17.004 0
1.179877785 -17.207 0
1.179877785 -17.304 0
1.179877785 -17.41 0
1.179877785 -17.515 0
1.179877785 -17.693 0
1.179877785 -17.781 0
1.179877785 -17.997 0
1.179877785 -18.101 0
1.179877785 -18.217 0
1.179877785 -18.316 0
1.179877785 -18.494 0
1.179877785 -18.71 0
1.179877785 -18.811 0
1.179877785 -18.909 0
1.179877785 -19.075 0
1.179877785 -19.193 0
1.179877785 -19.332 0
1.179877785 -19.41 0
1.179877785 -19.517 0
1.179877785 -19.648 0
1.179877785 -19.685 0
1.179877785 -20.1 0
1.179877785 -20.663 0
1.179877785 -20.663 0
1.179877785 -551.313 0
1.179877785 -557.356 0
WINNER: 2521913 with sp of None
